In [4]:
import os
import sys
sys.path.append('../../Project_Clustering')

import numpy as np
import pandas as pd
import pickle 
from matplotlib import cm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import myClusterPackage as myPack
from sklearn.mixture import GaussianMixture
from mySpectralClustering import spectralFeatures

In [2]:
face_db = []
face_list = []
labels = []
#seperator =','

data_folder = os.listdir("../3DFACEPAPER/Data/") # list of all subfolder within Data folder
data_folder.remove('.DS_Store') 


for k, expressions in enumerate (data_folder): # will loop within the main folder
   
    # Accessing each subfolder (expression) within the data folder  
    path =  "../3DFACEPAPER/Data/" + expressions +"/"
    face_names = os.listdir(path)    # list of all file names within subfolder
    
   
    for face in face_names: # will loop within the subfolder
        
        if not face.startswith('.'):
       # import the face and face labels
            file_name = path + face
        
            df_face = pd.read_csv(file_name)
            face_list.append (df_face.values)
            labels.append (expressions)
            

#pickle.dump(face_list, open('face_data','wb'))
#pickle.dump (labels, open('face_labels', 'wb'))


    

## Extracting point cloud features

In [2]:
face_list = pickle.load(open('face_data', 'rb'))
face_labels = pickle.load(open('face_labels', 'rb'))

all_patch_data = []
var_df = pd.DataFrame()
n_face = len(face_list)

km01 = KMeans(
        n_clusters=8, init='k-means++',
        n_init=10, max_iter=300, 
        tol=1e-04, random_state=10
    )

mvpk = np.where(face_list[0][:,2]== max(face_list[0][:,2]))
faceo = face_list[0] - face_list[0][mvpk]
km01 = km01.fit(face_list[0])
init_centroids = km01.cluster_centers_


for i,face in enumerate(face_list):
    
    
    km = KMeans(
        n_clusters=8, init=init_centroids,
        n_init=1, max_iter=300, 
        tol=1e-04, random_state=10
    )

    mvpk = np.where(face[:,2]== max(face[:,2]))
    
    face = face - face[mvpk[0][0]]

    segments = km.fit_predict(face)
    n_patch = len(np.unique(segments)) 
    centroids, _ = myPack.get_centroids(face, segments)
    total_features = []
   
   
    for j in range (n_patch):
        
        patch_features = []
        clusterLabels = []
        featureLabels = []
        
        patch = face[segments == j]
        delta_patch = patch - centroids[j] ## line of code to be added
        
        #transFeature = spectralFeatures (delta_patch,n_dim = 2, n_neighbor = 10)
        #df_2Col = transFeature.values
        #plt.plot(df_2Col[:,0],df_2Col[:,1])
        
        ## Applying GMM on each patch
        gmm = GaussianMixture(n_components = 3, covariance_type= 'diag',random_state=10)
        patch_gmm = gmm.fit(delta_patch)
        patch_w = patch_gmm.weights_
        patch_mu = patch_gmm.means_
        patch_sigma = patch_gmm.covariances_
        
        ## creating data frame for testing
        n_features = len(patch_w) + len(patch_mu)*len(patch_mu[1]) + len(patch_sigma)*len(patch_sigma[1])
    
        # Features extracted from a single patch
        # 3 + 6 + 6 --> repeated across 8 patches
        patch_features = list(patch_w) + list(patch_mu.ravel()) + list(patch_sigma.ravel())
        test_df = {'face_num':i,'cluster_labels':j , 'feature_labels':patch_features,'classes':face_labels[i]}
        var_df = var_df.append(test_df,ignore_index=True)
        
        # concatenating from all patches
        total_features = total_features + patch_features
    # adding features from all patches to the face database 
    all_patch_data.append (total_features)

df_3D_face = pd.DataFrame({'Patch_Features':all_patch_data,'Classes':face_labels})                                      

# Encoding class labels 
le = preprocessing.LabelEncoder() 
df_3D_face['Labels']= le.fit_transform(df_3D_face['Classes']) 
 

In [9]:
#df_3D_face

df_3D_face.to_pickle("all_face_TABLE_01.pkl")

#var_df

# Splitting the list into multiple PD columns
var_dfx = var_df.feature_labels.apply(pd.Series)

final_df= pd.concat([var_df,var_dfx],axis =1)
#final_df

var_feat = final_df.groupby('cluster_labels').var().reset_index()
var_feat.round(decimals = 3)

,cluster_labels,face_num,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,0.0,4.667,0.017,0.021,0.016,208.621,167.545,23.372,51.230,182.665,...,20.675,2768.665,8258.203,1101.145,1479.044,10611.212,713.791,3724.295,9144.936,952.192
1,1.0,4.667,0.001,0.002,0.000,44.808,169.830,164.717,100.586,163.670,...,110.421,1150.590,233.848,1441.939,1344.352,594.164,1145.086,237.723,1036.617,1878.893
2,2.0,4.667,0.011,0.006,0.016,94.754,67.247,30.948,90.853,110.221,...,42.236,316.290,5950.776,286.383,337.435,1906.773,322.765,631.880,7931.634,135.985
3,3.0,4.667,0.007,0.009,0.020,194.567,58.666,73.793,149.976,222.875,...,15.828,767.078,4709.597,279.575,539.075,2646.832,411.324,408.359,9174.279,132.288
4,4.0,4.667,0.001,0.003,0.003,79.329,234.207,3.831,82.559,187.246,...,191.773,376.042,205.922,677.886,1445.171,787.767,454.265,1137.482,308.564,4581.814
5,5.0,4.667,0.006,0.013,0.030,292.522,95.850,63.202,298.230,104.892,...,46.902,4083.994,103.412,1220.387,7023.692,757.415,1567.739,3184.072,3461.784,927.976
6,6.0,4.667,0.003,0.003,0.004,77.399,14.191,135.325,106.492,11.136,...,138.642,261.695,5998.512,91.556,375.536,10085.567,398.952,469.080,8073.888,62.088
7,7.0,4.667,0.001,0.001,0.003,52.536,22.207,117.054,93.033,16.402,...,306.120,244.807,6145.824,311.935,346.913,11517.091,428.248,350.857,342.271,425.820


In [17]:
var_data = var_feat.iloc[:, 2:]

var_w = var_data.iloc[:,0:3].round(decimals=3)

var_mu_0 = var_data.iloc[:,3:6].sum(axis=1).round(decimals=3)#.reset_index()

var_mu_1 = var_data.iloc[:,6:9].sum(axis=1).round(decimals=3)#.reset_index()

var_mu_2 = var_data.iloc[:,9:12].sum(axis=1).round(decimals=3)#.reset_index()

var_sigma_0 = var_data.iloc[:,12:15].sum(axis=1).round(decimals=3)#.reset_index()

var_sigma_1 = var_data.iloc[:,15:18].sum(axis=1).round(decimals=3)#.reset_index()

var_sigma_2 = var_data.iloc[:,18:21].sum(axis=1).round(decimals=3)#.reset_index()

gamma0 = [var_w[0].to_list(), var_mu_0.to_list(), var_sigma_0.to_list()] 
gamma1 = [var_w[1].to_list(), var_mu_1.to_list(), var_sigma_1.to_list() ]
gamma2 = [var_w[2].to_list(), var_mu_2.to_list(), var_sigma_2.to_list() ]

gamma2


[[0.016, 0.0, 0.016, 0.02, 0.003, 0.03, 0.004, 0.003],
 [413.946, 420.028, 232.474, 110.726, 351.298, 241.411, 244.894, 457.24],
 [13821.422,
  3153.233,
  8699.499,
  9714.926,
  6027.86,
  7573.832,
  8605.055,
  1118.947]]

In [7]:
#df_3D_face.to_pickle("df_3D_face_for_testing.pkl")
# Concatenation of the lists within a group
full_dfx  = var_df.groupby('face_num')['feature_labels'].sum().reset_index()

# Splitting the list into multiple PD columns
full_df = full_dfx.feature_labels.apply(pd.Series)

# Calculating feature variance 
fea_var = full_df.var()

# Constructing -- (n_faceXn_patch) x n_features -- array with variance
var_resh =np.reshape(list(fea_var)*n_face,(n_patch*n_face,n_features))

# Data frame of feature variance
feature_var = pd.DataFrame(data = var_resh,index=var_df.index)

# Concatenate with exiting data frame
final_df= pd.concat([var_df,feature_var],axis =1)
final_df


,classes,cluster_labels,face_num,feature_labels,0,1,2,3,4,5,...,11,12,13,14,15,16,17,18,19,20
0,Happy,0.0,0.0,"[0.29619595887990946, 0.488032258710453, 0.215...",0.016640,0.021345,0.015856,208.621322,167.544824,23.371967,...,20.675451,2768.665015,8258.203348,1101.144942,1479.043574,10611.212243,713.790701,3724.295067,9144.935711,952.191549
1,Happy,1.0,0.0,"[0.318634788114635, 0.35850254036087464, 0.322...",0.001048,0.001705,0.000381,44.808217,169.829503,164.717430,...,110.420678,1150.590360,233.848328,1441.939269,1344.352202,594.164349,1145.086158,237.723215,1036.616521,1878.893039
2,Happy,2.0,0.0,"[0.13963165101348624, 0.3577389771600363, 0.50...",0.011405,0.005957,0.015559,94.754109,67.247176,30.948359,...,42.236062,316.290112,5950.776305,286.382743,337.435469,1906.772733,322.765073,631.879795,7931.634283,135.984652
3,Happy,3.0,0.0,"[0.3217611048325352, 0.31519547866147535, 0.36...",0.007109,0.008676,0.020242,194.567353,58.665511,73.793063,...,15.828178,767.078359,4709.597072,279.575435,539.074791,2646.832017,411.324409,408.359303,9174.278634,132.287820
4,Happy,4.0,0.0,"[0.31752199054035546, 0.3932300710932048, 0.28...",0.001332,0.002910,0.003009,79.329276,234.206660,3.831212,...,191.773361,376.041751,205.922224,677.885987,1445.170728,787.766666,454.265326,1137.481849,308.564436,4581.813995
5,Happy,5.0,0.0,"[0.2273555580995069, 0.19834499215817805, 0.57...",0.006123,0.013409,0.030417,292.522082,95.849652,63.202038,...,46.901653,4083.994238,103.412353,1220.386710,7023.692026,757.415041,1567.738710,3184.071949,3461.784117,927.976136
6,Happy,6.0,0.0,"[0.41650130151642983, 0.25352038639901203, 0.3...",0.003109,0.002964,0.003650,77.398613,14.191450,135.325072,...,138.641691,261.695111,5998.512348,91.556473,375.536281,10085.566692,398.951630,469.079736,8073.887805,62.087922
7,Happy,7.0,0.0,"[0.39769917433459034, 0.3350844019428378, 0.26...",0.001080,0.001283,0.002676,52.535968,22.207386,117.054151,...,306.120253,244.807385,6145.824360,311.935136,346.913431,11517.091146,428.247943,350.856782,342.270634,425.819622
8,Sad,0.0,1.0,"[0.5744245801821405, 0.2527508229614497, 0.172...",0.016640,0.021345,0.015856,208.621322,167.544824,23.371967,...,20.675451,2768.665015,8258.203348,1101.144942,1479.043574,10611.212243,713.790701,3724.295067,9144.935711,952.191549
9,Sad,1.0,1.0,"[0.33788934346824867, 0.3604484581714642, 0.30...",0.001048,0.001705,0.000381,44.808217,169.829503,164.717430,...,110.420678,1150.590360,233.848328,1441.939269,1344.352202,594.164349,1145.086158,237.723215,1036.616521,1878.893039


In [9]:
var_feat = final_df.groupby('cluster_labels').sum().reset_index()
var_feat

,cluster_labels,face_num,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,0.0,21.0,0.116480,0.149413,0.110994,1460.349255,1172.813771,163.603771,358.611899,1278.658114,...,144.728154,19380.655108,57807.423434,7708.014597,10353.305021,74278.485701,4996.534907,26070.065467,64014.549979,6665.340842
1,1.0,21.0,0.007334,0.011934,0.002667,313.657521,1188.806518,1153.022010,704.099482,1145.688710,...,772.944747,8054.132517,1636.938295,10093.574882,9410.465412,4159.150441,8015.603103,1664.062503,7256.315645,13152.251271
2,2.0,21.0,0.079833,0.041697,0.108912,663.278764,470.730234,216.638513,635.970513,771.545300,...,295.652437,2214.030786,41655.434134,2004.679198,2362.048280,13347.409132,2259.355510,4423.158567,55521.439982,951.892566
3,3.0,21.0,0.049764,0.060730,0.141694,1361.971469,410.658576,516.551439,1049.835156,1560.125732,...,110.797245,5369.548510,32967.179502,1957.028046,3773.523535,18527.824118,2879.270863,2858.515119,64219.950438,926.014741
4,4.0,21.0,0.009324,0.020367,0.021060,555.304931,1639.446617,26.818482,577.915808,1310.719703,...,1342.413525,2632.292260,1441.455565,4745.201906,10116.195093,5514.366663,3179.857284,7962.372942,2159.951052,32072.697964
5,5.0,21.0,0.042859,0.093864,0.212919,2047.654575,670.947562,442.414266,2087.612475,734.241628,...,328.311568,28587.959664,723.886472,8542.706968,49165.844183,5301.905288,10974.170971,22288.503640,24232.488819,6495.832955
6,6.0,21.0,0.021763,0.020746,0.025552,541.790291,99.340148,947.275502,745.445827,77.955150,...,970.491838,1831.865774,41989.586438,640.895313,2628.753965,70598.966845,2792.661413,3283.558149,56517.214633,434.615456
7,7.0,21.0,0.007557,0.008978,0.018729,367.751779,155.451701,819.379055,651.232423,114.810820,...,2142.841768,1713.651694,43020.770517,2183.545955,2428.394018,80619.638021,2997.735600,2455.997472,2395.894438,2980.737356


In [ ]:
fea_var

In [ ]:
p_df = final_df.groupby('cluster_labels').mean().reset_index()
seg_int = p_df.iloc [:,2:].sum(axis=1)*1000
seg_int

In [ ]:
xx = df_3D_face['Patch_Features'].to_list() 

data_matrix = np.array (xx)

d_labels = df_3D_face['Labels'].to_list() 

print(data_matrix.shape)

In [ ]:
variances = np.var(data_matrix, axis=0)


fea_sel = np.where(variances>0.004)

print (fea_sel)
fig = plt.figure(figsize=(5,5))

plt.plot(range(len(variances)),variances, color='k',marker = 'o')
plt.xlabel ('Feature Index')
plt.ylabel ('Variance')

#save the feature indices (fea_sel) - 

#np.save('test3.npy', a)    # .npy extension is added if not given
#d = np.load('test3.npy')



In [ ]:

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import time

xx = df_3D_face['Patch_Features'].to_list() 

data_matrix = np.array (xx)

df = df_3D_face.copy()

# Performing PCA on raw data 
pca = PCA(n_components=5)
pca_result = pca.fit_transform(data_matrix)
df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(5,5))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="Classes",
    palette=sns.color_palette("hls", n_face),
    data=df,
    legend="full",
    alpha=1.0
)

# Performing t-SNE on raw data
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_matrix)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]

plt.figure(figsize=(5,5))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="Classes",
    palette=sns.color_palette("hls", n_face),
    data=df,
    legend="full",
    alpha=1.0
)

# Performing t-SNE on PCA data 
tsne_results = tsne.fit_transform(pca_result)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]

plt.figure(figsize=(5,5))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="Classes",
    palette=sns.color_palette("hls", n_face),
    data=df,
    legend="full",
    alpha=1.0
)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import re
import os

#path =  "../3DFACEPAPER/Data/Anger/" 

#face_names = os.listdir(path) 

#file_Path = path+face_names[2] 

#print (file_Path)
#print (face_names[2])
holder = []
with open("F0001_AN04WH_F3D.wrl", "rb") as vrml:
    for lines in vrml:
        a = re.findall("[-0-9]{1,3}.[0-9]{6}", lines) ## I think we need to work on this line
        if len(a) == 3:
            holder.append(map(float, a))

holder_array = np.array(holder) #if you want numpy array

#3D Plotting
x,y,z = zip(*holder)
fig = plt.figure()
ax = Axes3D(fig)
ax.plot(x,y,z)
plt.show()

In [ ]:
## for figure 02 in the table

df = pd.read_csv('../3DFACEPAPER/Data/Anger/F0001_AN04WH_F3D.csv')
df = pd.DataFrame({'X':df.iloc[:,0],'Y':df.iloc[:,1],'Z':df.iloc[:,2]})


In [ ]:
km01 = KMeans(
        n_clusters=8, init='k-means++',
        n_init=10, max_iter=300, 
        tol=1e-04, random_state=10
    )

mvpk = np.where(face_list[0][:,2]== max(face_list[0][:,2]))
faceo = face_list[0] - face_list[0][mvpk]
    
    
km01 = km01.fit(faceo)
init_centroids = km01.cluster_centers_

#colorset = ['green', 'orange', 'purple', 'blue', 'lightblue','red','yellow','red']
#markerset = ['s','d', 'o', 'v', 'p','d','1']
#intensity = (seg_int - seg_int.min() )/(seg_int.max()- seg_int.min())

#labels_int = np.zeros(segments01.shape,dtype=np.float64)
#for i in range (8):
    #labels_int[np.where(segments01 == i)] = intensity[i]

#%matplotlib tk
#fig02 = plt.figure(figsize=(15,8))
#ax = Axes3D(fig02)
#ax.scatter(df01[:,0], df01[:,1], df01[:,2] , c=labels_int ,marker='s', cmap ='Greens')
#plt.title('Segmentation of 3D facial point cloud')


In [ ]:
df = df.values
km = KMeans(
        n_clusters=8, init= init_centroids,
        n_init=1, max_iter=300, 
        tol=1e-04, random_state=10
    )
mvpk = np.where(df[:,2]== max(df[:,2]))
    
face = df - df[mvpk[0][0]]
segments = km.fit_predict(face)


In [ ]:
#df = df.values
%matplotlib tk
fig02 = plt.figure(figsize=(15,8))
ax = Axes3D(fig02)
for i in range(8):
    ax.scatter(face[segments==i,0],face[segments==i,1],face[segments==i,2],label='cluster'+str(i))
ax.legend(scatterpoints = 1)

In [2]:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [6]:
import numpy as np
import pandas as pd
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool

df = pd.DataFrame(np.random.randint(3, 10, size=[500, 2]))

def func(df):
    return df.shape

cores=mp.cpu_count()

df_split = np.array_split(df, cores, axis=0)

# create the multiprocessing pool
pool = Pool(cores)

# process the DataFrame by mapping function to each df across the pool
df_out = np.vstack(pool.map(func, df_split))

# close down the pool and join
pool.close()
pool.join()
pool.clear()

In [10]:
df_out

array([[63,  2],
       [63,  2],
       [63,  2],
       [63,  2],
       [62,  2],
       [62,  2],
       [62,  2],
       [62,  2]])